# Lack of Docstrings 

In [ ]:
from dask import array as da

In [8]:
import dask

In [7]:
?da

In [5]:
from numpy import array

In [6]:
?array

Basic Module summary

In [6]:
"""SparkSQL backend for blaze.

Notes
-----
Translation happens via the Hive sqlalchemy dialect, which is then sent to
SparkSQL.
"""

'SparkSQL backend for blaze.\n\nNotes\n-----\nTranslation happens via the Hive sqlalchemy dialect, which is then sent to\nSparkSQL.\n'

# Towards out of core NDArrays 

In [12]:
def inc(x):
    return x + 1

def add(x, y):
    return x + y

d = {'x': 1,
     'y': (inc, 'x'),
     'z': (add, 'y', 10)}

In [15]:
dask.get(d, 'y')

2

In [20]:
import numpy as np

In [21]:
x = np.arange(24).reshape((4,6))

In [22]:
x

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23]])

In [23]:
ndget

NameError: name 'ndget' is not defined

# Blaze Getting started

In [24]:
from blaze import Symbol

In [25]:
accounts = Symbol('accounts', 'var * {id: int, name:string, amount: int}')

In [26]:
?Symbol

# Understanding Functools

In [27]:
from functools import partial

In [28]:
basetwo = partial(int, base=2)

In [31]:
basetwo.__doc__ = 'Convert base 2 string to int'

In [32]:
basetwo('10001')

17

In [33]:
basetwo(1312)

TypeError: int() can't convert non-string with explicit base

In [1]:
x = Data(1)

NameError: name 'Data' is not defined

# Example 

In [2]:
from blaze import Data

In [3]:
t = Data([(1, 'Alice', 100),
           (2, 'Bob', -200),
           (3, 'Charlie', 300),
           (4, 'Denis', 400),
           (5, 'Edith', -500)],
          fields=['id', 'name', 'balance'])

In [5]:
type(t)

blaze.interactive.InteractiveSymbol

# Apply-Split-Combine Pandas

# Blah 

In [8]:
from __future__ import absolute_import, division, print_function

from datetime import datetime
from decimal import Decimal

import sqlalchemy as sa
import sqlalchemy.orm
from toolz import curry
from datashape.predicates import isrecord
from blaze.expr import Field
from odo.backends.sql import dshape_to_alchemy

# This was taken from the following StackOverflow post
# http://stackoverflow.com/questions/5631078/sqlalchemy-print-the-actual-query
# answer by bukzor http://stackoverflow.com/users/146821/bukzor


def literalquery(statement, dialect=None):
    """Generate an SQL expression string with bound parameters rendered inline
    for the given SQLAlchemy statement.

    WARNING: This method of escaping is insecure, incomplete, and for debugging
    purposes only. Executing SQL statements with inline-rendered user values is
    extremely insecure.
    """
    if isinstance(statement, sqlalchemy.orm.Query):
        if dialect is None:
            dialect = statement.session.get_bind(
                statement._mapper_zero_or_none()
            ).dialect
        statement = statement.statement
    if dialect is None:
        dialect = getattr(statement.bind, 'dialect', None)
    if dialect is None:
        from sqlalchemy.dialects import mysql
        dialect = mysql.dialect()

    Compiler = type(statement._compiler(dialect))

    class LiteralCompiler(Compiler):
        visit_bindparam = Compiler.render_literal_bindparam

        def render_literal_value(self, value, type_):
            if isinstance(value, (Decimal, long)):
                return str(value)
            elif isinstance(value, datetime):
                return repr(str(value))
            else:  # fallback
                value = super(LiteralCompiler, self).render_literal_value(
                    value, type_,
                )
                if isinstance(value, unicode):
                    return value.encode('UTF-8')
                else:
                    return value

    return LiteralCompiler(dialect, statement)


In [19]:
literalquery("test", HiveDialect())

NameError: name 'HiveDialect' is not defined

In [17]:
!pwd

/Users/Will/Devel/blaze-tutorial


In [16]:
from toolz import pipe
from toolz.curried import filter, map

from blaze.dispatch import dispatch
from blaze.expr import Expr, symbol
from blaze import compute
from blaze.compute.spark import Dummy
from blaze.compute.utils import literalquery, istable, make_sqlalchemy_table


__all__ = []


try:
    from pyspark import SQLContext
    from pyhive.sqlalchemy_hive import HiveDialect
except ImportError:
    SQLContext = Dummy


@dispatch(Expr, SQLContext)
def compute_down(expr, data, **kwargs):
    """ Compile a blaze expression to a sparksql expression"""
    leaves = expr._leaves()

    # make sure we only have a single leaf node
    if len(leaves) != 1:
        raise ValueError('Must compile from exactly one root database')

    leaf, = leaves

    # field expressions on the database are Field instances with a record
    # measure whose immediate child is the database leaf
    tables = pipe(expr._subterms(), filter(istable(leaf)), list)

    # raise if we don't have tables in our database
    if not tables:
        raise ValueError('Expressions not referencing a table cannot be '
                         'compiled')

    # make new symbols for each table
    new_leaves = [symbol(t._name, t.dshape) for t in tables]

    # sub them in the expression
    expr = expr._subs(dict(zip(tables, new_leaves)))

    # compute using sqlalchemy
    scope = dict(zip(new_leaves, map(make_sqlalchemy_table, tables)))
    query = compute(expr, scope)

    # interpolate params
    compiled = literalquery(query, dialect=HiveDialect())
    return data.sql(str(compiled))


ImportError: cannot import name istable